# Competition-Compliant Ensemble Notebook

**CRITICAL**: This notebook follows the EXACT template structure required by the competition.
- Last 3 cells are IDENTICAL to the template
- Only the model definition line is changed
- Model class has `train_model(X_train, y_train)` and `predict(X)` methods

**Model**: SimpleEnsemble = [32,16] MLP (0.6) + LightGBM (0.4)
**Best CV**: 0.008785 from exp_011

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import tqdm
import warnings
import sys
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Data loading functions (matching template)
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

In [ ]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

In [ ]:
# Combined Featurizer with Arrhenius kinetics
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

In [ ]:
# MLP Model [32,16] with BatchNorm
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLPModelInternal defined')

In [ ]:
# MLP Ensemble (5 models with different seeds)
class MLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single'):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            
            model = MLPModelInternal(input_dim, hidden_dims=self.hidden_dims, dropout=0.05).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize_torch(X, flip=False).to(device)
            X_flip = self.featurizer.featurize_torch(X, flip=True).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize_torch(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models
        return avg_pred.cpu()

print('MLPEnsemble defined')

In [ ]:
# LightGBM Model
class LGBMModel:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []
        self.params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1,
            'seed': 42
        }

    def train_model(self, X_train, y_train, num_boost_round=200):
        X_feat = self.featurizer.featurize(X_train)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all = X_feat
            y_all = y_train.values
        
        self.models = []
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(self.params, train_data, num_boost_round=num_boost_round)
            self.models.append(model)

    def predict(self, X):
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False)
            X_flip = self.featurizer.featurize(X, flip=True)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = (model.predict(X_std) + model.predict(X_flip)) / 2
        else:
            X_feat = self.featurizer.featurize(X)
            preds = np.zeros((len(X), 3))
            for i, model in enumerate(self.models):
                preds[:, i] = model.predict(X_feat)
        return torch.tensor(np.clip(preds, 0, 1))

print('LGBMModel defined')

In [ ]:
# SimpleEnsemble: [32,16] MLP (0.6) + LightGBM (0.4)
# This is the model class that will be used in the template cells
class SimpleEnsemble:
    """Competition-compliant ensemble model.
    
    Has train_model(X_train, y_train) and predict(X) methods.
    Accepts data='single' or data='full' parameter.
    """
    def __init__(self, data='single', mlp_weight=0.6, lgbm_weight=0.4):
        self.data_type = data
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.mlp = None
        self.lgbm = None

    def train_model(self, X_train, y_train):
        """Train both MLP and LightGBM models."""
        # Train MLP [32,16] - best LB model
        self.mlp = MLPEnsemble(hidden_dims=[32, 16], n_models=5, data=self.data_type)
        self.mlp.train_model(X_train, y_train, epochs=200)
        
        # Train LightGBM
        self.lgbm = LGBMModel(data=self.data_type)
        self.lgbm.train_model(X_train, y_train, num_boost_round=200)

    def predict(self, X):
        """Return weighted average of MLP and LightGBM predictions."""
        pred_mlp = self.mlp.predict(X)
        pred_lgbm = self.lgbm.predict(X)
        
        # Weighted average
        ensemble_pred = self.mlp_weight * pred_mlp + self.lgbm_weight * pred_lgbm
        return torch.clamp(ensemble_pred, 0, 1)

print('SimpleEnsemble defined: MLP[32,16] (0.6) + LightGBM (0.4)')
print('Model interface: train_model(X_train, y_train), predict(X)')

---
## COMPETITION TEMPLATE CELLS BELOW

**The following 3 cells are EXACTLY as required by the competition template.**
**Only the model definition line is changed.**

---

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SimpleEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SimpleEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################